In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.multiprocessing as mp
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset
import torch.optim as optim
from torch.cuda.amp import  GradScaler
from torch.amp import autocast
import gc
from dataset_convert import AnchorGenerator, FaceDetectionDataset
# device = torch.device("mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/faces2.0/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /home/.cache/torch/hub/pytorch_vision_v0.10.0
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/faces2.0/.conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

In [4]:
extractor = Features(model,['3','6', '13','18'])
topdown = FPNetwork(out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction(lambd=10)
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [ ]:
# (extractor.state_dict(),'/home/faces2.0/models/extractor.pt')
# torch.save(topdown.state_dict(),'/home/faces2.0/models/topdown.pt')
# torch.save(classifier.state_dict(),'/home/faces2.0/models/classifier.pt')
# torch.save(bboxregression.state_dict(),'/home/faces2.0/models/bboxregression.pt')

In [47]:
sd_extractor = torch.load('/home/faces2.0/models/extractor.pt',map_location=torch.device('cpu'))
sd_classifier = torch.load('/home/faces2.0/models/classifier.pt',map_location=torch.device('cpu'))
sd_bboxregression = torch.load('/home/faces2.0/models/bboxregression.pt',map_location=torch.device('cpu'))
sd_topdown = torch.load('/home/faces2.0/models/topdown.pt',map_location=torch.device('cpu'))

extractor.load_state_dict(sd_extractor)
topdown.load_state_dict(sd_topdown)
classifier.load_state_dict(sd_classifier)
bboxregression.load_state_dict(sd_bboxregression)


<All keys matched successfully>

In [33]:
def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
    return output

In [55]:
class Forward:
    def __init__(self, extractor, topdown, bboxregression, classifier):
        self.extractor=extractor
        # self.extractor = Features(extractor,['3','6', '13','18'])
        self.topdown = topdown
        self.bboxregression = bboxregression
        self.classifier = classifier
        
    def __call__(self, inp):
        self.raw_features = self.extractor.extract(inp)
        self.newfeatures = self.topdown(self.raw_features)
        output = {}
        for key in list(self.newfeatures.keys()):
            temp = {}
            temp["bbox"] = self.bboxregression(self.newfeatures[key])
            temp["cls"] = self.classifier(self.newfeatures[key])
            output[key] = temp
            self.output = output
        return self.output        

In [60]:
fd= Forward(extractor=extractor,
            topdown=topdown,
            bboxregression=bboxregression,
            classifier=classifier
            )

In [57]:
inp= data[200][0].unsqueeze(0).cuda()

In [58]:
loss = Lossfunction()

In [61]:
s = fd(inp)
kk=data[200][1]

In [62]:
for key in range(0,4):
    print(kk[key]['cls_targets'].shape)
    print(kk[key]['cls_targets'].sum())

torch.Size([4800, 1])
tensor(0)
torch.Size([19200, 1])
tensor(12)
torch.Size([76800, 1])
tensor(69)
torch.Size([307200, 1])
tensor(178)


In [63]:
(torch.sigmoid(s['3']['cls']))

tensor([[[0.5000],
         [0.5000],
         [0.5000],
         ...,
         [0.4986],
         [0.4986],
         [0.4986]]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [64]:
loss(s, kk)

box loss:0.4609232246875763
cls loss:0.38423848152160645


tensor(0.8452, device='cuda:0', grad_fn=<AddBackward0>)

In [67]:
loss(s, data[0][1])


box loss:0.7665533423423767
cls loss:7.646162986755371


tensor(8.4127, device='cuda:0', grad_fn=<AddBackward0>)

In [45]:
for i in list(fd.raw_features.keys()):
    print (fd.raw_features[i].shape)
    print (fd.raw_features[i].mean())
    

torch.Size([1, 24, 160, 160])
tensor(0.0034, grad_fn=<MeanBackward0>)
torch.Size([1, 32, 80, 80])
tensor(-0.0042, grad_fn=<MeanBackward0>)
torch.Size([1, 96, 40, 40])
tensor(0.0060, grad_fn=<MeanBackward0>)
torch.Size([1, 1280, 20, 20])
tensor(0.2237, grad_fn=<MeanBackward0>)


In [47]:
for i in list(fd.newfeatures.keys()):
    print (fd.newfeatures[i].shape)
    print (fd.newfeatures[i].mean())

torch.Size([1, 256, 20, 20])
tensor(-0.1097, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 40, 40])
tensor(-0.1092, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 80, 80])
tensor(-0.1187, grad_fn=<MeanBackward0>)
torch.Size([1, 256, 160, 160])
tensor(-0.1175, grad_fn=<MeanBackward0>)


In [42]:
loss(kk, data[0][1])


box loss:0.13568367063999176
cls loss:62838.42578125


tensor(62838.5625, grad_fn=<AddBackward0>)

In [52]:
model(inp).mean()

tensor(0.5135, grad_fn=<MeanBackward0>)